In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp001_t5mut_maxmask2/20iter_temp001_t5mut_maxmask2-mcmc_seqs.tsv'

In [3]:
perturb_results_tsvs = [
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01_t5mut_maxmask2/20iter_temp01_t5mut_maxmask2-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp001_t5mut_maxmask2/20iter_temp001_t5mut_maxmask2-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp01_t5mut_maxmask2/100iter_temp01_t5mut_maxmask2-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp001_t5mut_maxmask2/100iter_temp001_t5mut_maxmask2-mcmc_seqs.tsv',
#     'xxx',
#     'xxx',
#     'xxx',
   
]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='disc_pred', ascending=False)
noperturb_df

,disc_pred,gt_class_pred,MT_edit_dist_vs_WT,generated_seq_ppl,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq,accepted
0,11.423256,2,8,42.497021,0.999872,1,0.000038,0.000090,The dramatic crisis doesn't always succeed in ...,0
1,11.423103,2,13,41.985306,0.510228,1,0.000210,0.489561,If you keep your eyes open amid all the blood ...,0
2,11.410098,2,7,44.176228,0.999911,1,0.000037,0.000051,The dramatic crisis doesn't always succeed in ...,0
3,11.394756,2,7,51.081310,0.999927,1,0.000033,0.000040,The dramatic crisis doesn't always succeed in ...,0
4,11.394699,3,8,38.783234,0.979939,1,0.000140,0.979963,The dramatic crisis doesn't always play out in...,0
...,...,...,...,...,...,...,...,...,...,...
25979,-11.191644,0,4,86.592117,0.999922,0,0.999953,0.000033,"Well, it's as pathetic as a pet Animal.",1
25980,-11.235039,0,11,75.214531,0.995078,0,0.999867,0.000070,"The movie is lacking its virtues, and despite ...",0
25981,-11.241712,1,7,462.407684,0.999950,0,0.999954,0.000009,Most of the movies have come off as ghattomime...,0
25982,-11.247304,0,5,77.887253,0.999905,0,0.999953,0.000034,"Well, that's as pathetic as a pet Animal.",0


In [6]:
print("np.max(noperturb_df['generated_seq_ppl']): ", np.max(noperturb_df['generated_seq_ppl']))
print("np.min(noperturb_df['generated_seq_ppl']): ", np.min(noperturb_df['generated_seq_ppl']))
print("np.mean(noperturb_df['generated_seq_ppl']): ", np.mean(noperturb_df['generated_seq_ppl']))


np.max(noperturb_df['generated_seq_ppl']):  144473.828125
np.min(noperturb_df['generated_seq_ppl']):  3.8254857063293457
np.mean(noperturb_df['generated_seq_ppl']):  211.95875289887437


In [7]:
# noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
# noperturb_df

# Compute % target class for noperturb

In [8]:
target_classes = [3,4]
topk_list = [10000, 1000, 100, 10]

percent_target_class = []

In [9]:
gt_class_preds = noperturb_df['gt_class_pred']
# gen_input_seq_classes = noperturb_df['gen_input_seq_class']
# sent_deltas = noperturb_df['sent_delta']

In [10]:
df = noperturb_df

# iterate through all perturbed result tsv files

In [11]:
perturb_results_tsvs

['generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01_t5mut_maxmask2/20iter_temp01_t5mut_maxmask2-mcmc_seqs.tsv',
 'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp001_t5mut_maxmask2/20iter_temp001_t5mut_maxmask2-mcmc_seqs.tsv',
 'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp01_t5mut_maxmask2/100iter_temp01_t5mut_maxmask2-mcmc_seqs.tsv',
 'generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_100iter_temp001_t5mut_maxmask2/100iter_temp001_t5mut_maxmask2-mcmc_seqs.tsv']

In [12]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='disc_pred', ascending=False)
#     perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']

    gt_class_preds = perturb_df['gt_class_pred']
#     gen_input_seq_classes = perturb_df['gen_input_seq_class']
#     sent_deltas = perturb_df['sent_delta']
    generated_seq_ppls = perturb_df['generated_seq_ppl']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
#             topk_sent_deltas = sent_deltas[:topk]


            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

#             topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#             print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
#         topk_sent_deltas = sent_deltas[:topk]
    
        topk_generated_seq_ppls = generated_seq_ppls[:topk]


        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
        
#         topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#         print("topk_sent_delta_mean: ", topk_sent_delta_mean)

        print("*")
    
    
    
    
    
        
    # E[% positive, strong-positive] computation

    df = perturb_df

    num_rounds = 100 # N
    round_pool_size = 1000
    topk = 100 # K
    main_pool_size = 25000
    target_classes = [3, 4]

    round_topk = {}
#     cols_to_sort = ['latent_head_pred']
    cols_to_sort = ['disc_pred']
    df_main_pool = df.sample(n=main_pool_size)


    print("--------------")
    print("E[% positive, strong-positive] computation")
    # print("Sorted by ", cols_to_sort)
    for col_to_sort in cols_to_sort:
        print("col_to_sort: ", col_to_sort)
        round_topk[col_to_sort] = {}
        for round_ind in range(num_rounds):
            sampled_rows = df_main_pool.sample(n=round_pool_size)
            sorted_sampled_rows = sampled_rows.sort_values(by=col_to_sort, ascending=False)[:topk]
            topk_rows = sorted_sampled_rows[:topk]
            round_topk[col_to_sort][round_ind] = {}



            for target_class in target_classes:
                total_num  = len(topk_rows['gt_class_pred'])
    #             print("target_class: ", target_class)
                num_target_class = np.sum(topk_rows['gt_class_pred'] == target_class)

                percent_target_class = num_target_class / total_num *100
    #             print("percent_target_class: ", percent_target_class)

                round_topk[col_to_sort][round_ind][target_class] = percent_target_class



    #         print("target_classes: ", target_classes)
            total_num  = len(topk_rows['gt_class_pred'])
            num_target_class = np.sum(topk_rows['gt_class_pred'].isin(target_classes))

            percent_target_class = num_target_class / total_num *100
    #         print("percent_target_class: ", percent_target_class)
            round_topk[col_to_sort][round_ind]['all'] = percent_target_class


        for target_class in target_classes:
            percent_values = []
            for round_ind in range(num_rounds):
                percent_values.append(round_topk[col_to_sort][round_ind][target_class])
            print("target_class: ", target_class)
            mean_percent_values = np.mean(percent_values)
            std_percent_values = np.std(percent_values)
            print("mean_percent_values: ", mean_percent_values)
            print("std_percent_values: ", std_percent_values)



        percent_values = []
        for round_ind in range(num_rounds):
            percent_values.append(round_topk[col_to_sort][round_ind]['all'])
        print("target_classes: ", target_classes)
        mean_percent_values = np.mean(percent_values)
        std_percent_values = np.std(percent_values)
        print("mean_percent_values: ", mean_percent_values)
        print("std_percent_values: ", std_percent_values)

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/mcmc_SST5/SST5_mcmc_leave34out_trainlabel2initseqs_20iter_temp01_t5mut_maxmask2/20iter_temp01_t5mut_maxmask2-mcmc_seqs.tsv
target_class:  3
percent_target_class:  12.526939655172415
topk:  10000
topk_percent_target_class:  22.06
*
topk:  1000
topk_percent_target_class:  33.4
*
topk:  100
topk_percent_target_class:  23.0
*
topk:  10
topk_percent_target_class:  20.0
*
--------------
target_class:  4
percent_target_class:  1.3469827586206895
topk:  10000
topk_percent_target_class:  2.96
*
topk:  1000
topk_percent_target_class:  2.5
*
topk:  100
topk_percent_target_class:  0.0
*
topk:  10
topk_percent_target_class:  0.0
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  13.873922413793101
topk:  10000
topk_percent_target_class:  25.019999999999996
topk_generated_seq_ppl_mean:  280.77381875154924
topk_generated_seq_ppl_std:  4042.328161087036
*
